In [1]:
# final run
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()  # ✅ Ensure window is closed

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()  # ✅ Fully close the window
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Cleanup after loop ends
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Fully close after use
    return result

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set as the only selected tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Write filtered data
                filtered_df = df[df[split_column] == val]
                for i, row in enumerate(filtered_df.values, start=2):
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=i, column=j).value = value

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
#                 messagebox.showinfo("Done", f"Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Force close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\London.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Madrid.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Lisbon.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Berlin.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Paris.xlsx
